### **1.Imort liabries and data**

In [1]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 8.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
import emoji
from bs4 import BeautifulSoup

In [3]:
train = pd.read_csv("/content/drive/MyDrive/Big Data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Big Data/test.csv")
evaluation = pd.read_csv("/content/drive/MyDrive/Big Data/evaluation.csv")

In [4]:
#check the general display of data
train.head()

,score,text
0,0,"overgeneralized, not helpful to anyone serious..."
1,1,Great sound and service.
2,1,love this book!!!: this book is a fast read ab...
3,1,A hugely enjoyable screen version of Rona Jaff...
4,0,What an uninteresting hodge-podge. It could ha...


In [5]:
#Shape of all data
print(train.shape)
print(test.shape)
print(evaluation.shape)

(7500, 2)
(2500, 2)
(5000, 2)


### **2.Data Cleaning**

As i know from the file，Some data may contain HTML tags, hashtags, mentions, emojis, etc.

So i need to start solving these steps by steps

In [6]:
#Combine the data together,so we can continue all the process
all_data = pd.concat([train,test,evaluation])
all_data.shape

(15000, 2)

In [7]:
# Define preprocessing function
def preprocessing(text):
    # Remove HTML tags
    # https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=get_text#get-text
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text(strip=True)

    # Remove Hashtags、Mentions and URL
    # https://docs.python.org/3/library/re.html#functions
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'http\S+', '', text)

    # Remove repeated characters (e.g., "sooo cooool" -> "so cool")
    text = re.sub(r'(.)\1+', r'\1', text)

    #Remove Emojis
    #https://carpedm20.github.io/emoji/docs/#replacing-and-removing-emoji
    text = emoji.replace_emoji(text, replace='')

    text = text.lower()

    return text.strip()

# use the function
all_data['text']=all_data['text'].apply(preprocessing)

<ipython-input-7-6ff6ba665310>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [8]:
all_data.shape

(15000, 2)

In [9]:
#Transform 0 to negative，1 to positive
mapping = {
    0: 'negative',
    1: 'positive',
}

all_data['score'] = all_data['score'].map(mapping)

# save to a new csv file for further check
all_data.to_csv('processed_file.csv', index=False)

In [10]:
#Divide back to train，test，evaulation
train = all_data.iloc[:7500]
test = all_data.iloc[7500:10000]
evaluation = all_data.iloc[10000:]

train.shape,test.shape,evaluation.shape

((7500, 2), (2500, 2), (5000, 2))

In [11]:
train.to_csv('train.csv')

### **3.Model Building**

#### **3.1 Naïve Bayesian Classifier**

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train['text'])
X1 = vectorizer.transform(test['text'])

y = train['score']
y1 = test['score']

# Train the model
nb_model = MultinomialNB(alpha=0.3)
nb_model.fit(X, y)

# Predict and evaluate
y_pred = nb_model.predict(X1)
accuracy = accuracy_score(y1, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 83.00%


#### **3.2 SVM**

In [13]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear')
svm_model.fit(X, y)

y_pred = svm_model.predict(X1)
accuracy = accuracy_score(y1, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 84.72%


### **4.Evaluation**

In [14]:
# Naïve Bayesian Evaluation
X2 = vectorizer.transform(evaluation['text'])
y2 = evaluation['score']

y_pred1 = nb_model.predict(X2)
accuracy1 = accuracy_score(y2, y_pred1)
print(f"Model Accuracy: {accuracy1 * 100:.2f}%")

Model Accuracy: 82.60%


In [15]:
# SVM Evaluation
y_pred1 = svm_model.predict(X2)
accuracy1 = accuracy_score(y2, y_pred1)
print(f"Model Accuracy: {accuracy1 * 100:.2f}%")

Model Accuracy: 85.02%
